In [10]:
import numpy as np
import serial
import datetime as dt
import os
from os.path import join, exists
import time
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Process, Queue, Event
import glob
import pandas as pd
import warnings
import sys

import subprocess
from tqdm.notebook import tqdm
import datetime

#if you want to display images as you record
import cv2
import matplotlib.pyplot as plt

from pyk4a import *

# imports from this module
from top_bottom_triggered.fast_animate import *
from top_bottom_triggered.commutator_utils import *
from top_bottom_triggered.video_io import *
from top_bottom_triggered.multicam_utils import *


## File name + dir, expt length

In [11]:
subject = 'tmp'
time_in_minutes = 0.5 # slightly longer than mkv to ensure complete overlap
base_path = R'D:\Jonah\Thermistor_recordings'
# base_path = R'E:\Jonah\CeAMouse'
file_suffix = ''  # disambiguate between two sessions on the same day
date = dt.datetime.now().strftime('%Y%m%d')


In [13]:
overwrite = True
path = os.path.join(base_path, f'{subject}\\{date}_{subject}')
# path = path.format(subject=subject, date=date)

if not os.path.exists(path):
    os.makedirs(path)
    print(f'Created {path}')
else:
    print(f'Path {path} exists!')

Path D:\Jonah\Thermistor_recordings\tmp\20230526_tmp exists!


# Commutator Setup

In [14]:
commutator_port = 'COM4'


In [15]:
show_opto = False  # only set to true if there is a "stim" col in ino data
debug = False

In [16]:
# test serial port and check dac value
with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
    line = ino.readline().decode('utf-8').strip('\r\n')
    print(line)
    print(f'Data has {len(line.split(","))} elements')

1.5000,0.0000,-88.7500,-0.02,-0.36,0.02,0,3.30
Data has 8 elements


In [17]:
# hard-coded params -- don't chage
n_samples = 4000  # how many thermistor samples to show
q_downsample = 15  # leave at 15; how much to downsample rt output (doesnt affect saved data) (eg if 20, and ino at 1 khz, will be 50 hz)
fs = 500  # fs of the commutator teensy
commutator_fname = f'{date}_{subject}{file_suffix}.txt'
commutator_fullfile = os.path.join(path, commutator_fname)
if exists(commutator_fullfile) and not overwrite:
    raise ValueError(f'File {commutator_fullfile} exists! Add a suffix or change subject name.')
elif exists(commutator_fullfile):
    os.remove(commutator_fullfile)
else:
    pass

## Prep the experiment!
(One priming cell)

In [18]:
# Get the header from the arduino, and save it to the file
first_line = 1  # don't change
second_line = 0  # don't change
sync_sent = 0
header_max_attempts = 10
second_line_max_attempts = 10

# Open queue to animator
thermistor_animator = ThermistorAnimator(n_samples, 500, show_opto=show_opto)

with open(commutator_fullfile, 'x') as file:
    with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
        ino.write('r'.encode('utf-8')) ## reset trigger counter
        reader = ReadLine(ino)
        
        # These checks get header and process it
        if first_line:
            # Ask the arduino to print the header
            ino.write('h'.encode('utf-8'))

            # Verify first line. First_line becomes false when good (ie, we're no longer on the first line)
            status, first_line, second_line, header, n_attempts, read_lines = first_line_check(header_max_attempts, reader, file=file)
            if not status:
                raise RuntimeError('Didnt receive header!')

            # Extract indices of values we're intersted in
            header_len = len(header.split(','))
            print(header)
            thermistor_animator.extract_indices_from_header(header)
        
        if second_line:
            status, second_line = second_line_check(second_line_max_attempts, reader, header, n_good_thresh=10)
        if not status:
                raise RuntimeError('Number of csv''d datapoints doesnt match number of csv''d elements in header!') 
                
print('Header looks good!')

time,led1,led2,led3,led4,yaw,roll,pitch,acc_x,acc_y,acc_z,therm,dac


IndexError: list index out of range

## Run the experiment!

In [9]:
# timing vars
start_time = dt.datetime.now()
one_mindelta = dt.timedelta(minutes=1)
exp_timedelta = time_in_minutes*one_mindelta # key var to be compared against (now - start_time)

# Main DAQ loop
try:
    with open(commutator_fullfile, 'a') as file:
        with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
            
            # Create more efficient serial reader
            reader = ReadLine(ino)

            # Start thermistor animator
            thermistor_animator.start()
            
            # Run the experiment for the requested amt of time!
            while (dt.datetime.now() - start_time) < exp_timedelta:  
                
                # Read the current line of data
                line = reader.readline().decode('utf-8').strip('\r\n')
                
                # Remove the DEBUG output if present and debugging
                if debug:
                    line = line[:(line.find(',DEBUG:'))]  
                    
                # Check for the typical (but relatively infrequent) serial read issues
                if len(line) == 0:
                    print('Got empty line, continuing...')
                    continue
                elif len(line.split(',')) != header_len:
                    print('Got line with unexpected length (skipping):')
                    print(line)
                    continue
                      
                # Write data to file
                file.write(line)
                file.write('\n')
                    
                # Update the animator
                thermistor_animator.update(line)
                
                # Test the exception handling
#                 if trigger_started_event.is_set():
#                     raise RuntimeError('test')
                    
            # After data collection finishes, close animator queue
            print('closing animator queue')
            thermistor_animator.close()
        
            print('Done with main loop')
            
# Catch unexpected errors            
except:
    
    # Stop the animator process
    print('Stopping animator')
    thermistor_animator.close()
    raise
    
finally:
    print('Done.')

closing animator queue
Done with main loop
Done.


## Post-experiment summaries

In [65]:
data = pd.read_csv(glob.glob(os.path.join(path, '*.txt'))[0])

In [66]:
therm_over_thresh_count = ((data.therm > 900) & (data.dac<=0.1)).sum()
therm_under_thresh_count = ((data.therm < 200) & (data.dac >= 3.25)).sum()
print(f'Therm over: {therm_over_thresh_count}')
print(f'Therm under: {therm_under_thresh_count}')
print(f'Time elapsed since start: {(dt.datetime.now() - start_time).seconds/60:0.1f} minutes')

Therm over: 4608
Therm under: 0
Time elapsed since start: 66.8 minutes
